# Goodbooks-10k Collaborative Genre Tagging
  
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/James-Leslie/deep-collaborative-filtering/blob/master/tf-goodbooks.ipynb)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

%matplotlib inline

## Load rating data

In [2]:
path = 'data/goodbooks-10k/'  # if the files are local
df = pd.read_csv(path+'ratings.csv')

In [3]:
df.head()

,user_id,book_id,rating
0,0,257,5
1,1,4080,4
2,1,259,5
3,1,9295,5
4,1,2317,3


In [4]:
df.shape

(5976479, 3)

In [5]:
print('Number of users:', df.user_id.nunique())
print('Number of items:', df.book_id.nunique())
print("Min item rating:", df.rating.min())
print("Max item rating:", df.rating.max())
print("Mean item rating:", df.rating.mean())

Number of users: 53424
Number of items: 10000
Min item rating: 1
Max item rating: 5
Mean item rating: 3.9198655261735214


## Load book metadata
  - remove 10% as holdout test set

In [6]:
books = pd.read_csv(path+'books.csv').fillna(0)
books.iloc[:,-10:] = books.iloc[:,-10:].astype('int')
books.head()

,book_id,original_title,language_code,adult,adult-fiction,adventure,book-club,contemporary,fantasy,fiction,mystery,romance,young-adult
0,0,The Hunger Games,eng,0,0,1,1,1,1,1,0,1,1
1,1,Harry Potter and the Philosopher's Stone,eng,0,0,1,0,1,1,1,1,0,1
2,2,Twilight,en-US,0,0,0,1,1,1,1,0,0,1
3,3,To Kill a Mockingbird,eng,1,1,0,1,1,0,1,1,0,1
4,4,The Great Gatsby,eng,1,1,0,1,0,0,1,0,1,1


In [7]:
from sklearn.model_selection import train_test_split

In [8]:
books, books_holdout = train_test_split(books, test_size=.1, random_state=42)

---
# Create baseline features
For each user, calculate average user bias - the average difference between the user's rating and the movie's average rating:

$$b_{u} = \dfrac{\sum_{j=1}^{n_u} (r_{uj} - \mu_i)}{n_u}$$

For each item, calculate the difference between its average rating and the average rating of all movies:

$$b_{i} = \dfrac{\sum_{k=1}^{n_i} (r_{ki})}{n_i} - \mu$$

Then, for each interaction, calculate the combined bias:

$$b_{ui} = \dfrac{b_u + b_i}{2}$$

In [9]:
from CGT import get_baseline
?get_baseline

Signature: get_baseline(df, train_index, test_index, user_col, item_col)
Docstring:
Calculate baseline features from an explicit ratings dataset. Receives a dataframe
and returns train and test splits with added bias column and mean rating value.
User and item biases are calculated as average difference from global mean rating.
Baseline factors are only calculated from training observations, with users or
items that do not appear in train receiving the global average as default.

Args:
    df          : explicit ratings dataframe with columns userId, movieId and rating
    train_index : train index splits taken from KFold.splits()
    test_index  : test index splits taken from KFold.splits()
    
Returns:
    train, test : train/test splits of df, with added bias column
    global_mean : average rating of all training observations
File:      c:\users\jleslie\documents\deep-collaborative-filtering\cgt.py
Type:      function


---
# CGT model
**To do**:
  - Can we avoid re-training rating model on CV fold?
  - Create a grid search function / class

In [10]:
from CGT import compile_genre_model
?compile_genre_model

Signature:
compile_genre_model(
    n_items,
    n_users,
    min_rating,
    max_rating,
    mean_rating,
    n_latent,
    n_hidden_1,
    n_hidden_2,
    activation='relu',
    dropout_1=0.2,
    dropout_2=0.2,
    random_seed=42,
)
Docstring: <no docstring>
File:      c:\users\jleslie\documents\deep-collaborative-filtering\cgt.py
Type:      function


# Classification report

In [11]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

In [12]:
books.head()

,book_id,original_title,language_code,adult,adult-fiction,adventure,book-club,contemporary,fantasy,fiction,mystery,romance,young-adult
4896,4896,Second Grave on the Left,en-US,1,1,0,0,1,1,1,1,1,0
4782,4782,Метро 2033,ger,1,0,1,0,1,1,1,1,0,0
1496,1496,The Borrowers Afield,0,0,0,1,0,0,1,1,0,0,1
1957,1957,Destined,eng,0,0,0,0,0,1,1,0,1,1
9171,9171,アンダーグラウンド [Andāguraundo],eng,1,0,0,1,1,0,1,0,0,0


In [13]:
books.shape

(9000, 13)

In [14]:
# get baseline predictors for full dataset
train, _, _ = get_baseline(df, df.index, df.index, 'user_id', 'book_id')

# compile both models
model1, model2 = compile_genre_model(
    n_items=df.book_id.nunique(),
    n_users=df.user_id.nunique(),
    min_rating=df.rating.min(),
    max_rating=df.rating.max(),
    mean_rating=train.rating.mean(),
    n_latent=25,
    n_hidden_1=25,
    n_hidden_2=25,
    dropout_1=.2,
    dropout_2=.2
)

In [15]:
# train rating model
ratings = model1.fit(
    x=[train.user_id.values, train.book_id.values, train.bias.values],
    y=train.rating.values, 
    batch_size=2048,
    epochs=6,
    verbose=1,
    validation_split=.2
)

Train on 4781183 samples, validate on 1195296 samples
Epoch 1/6
4781183/4781183 [==============================] - 44s 9us/sample - loss: 0.7451 - val_loss: 0.7323
Epoch 2/6
4781183/4781183 [==============================] - 45s 9us/sample - loss: 0.7125 - val_loss: 0.7369
Epoch 3/6
4781183/4781183 [==============================] - 45s 9us/sample - loss: 0.6896 - val_loss: 0.7412
Epoch 4/6
4781183/4781183 [==============================] - 46s 10us/sample - loss: 0.6713 - val_loss: 0.7428
Epoch 5/6
4781183/4781183 [==============================] - 47s 10us/sample - loss: 0.6578 - val_loss: 0.7482
Epoch 6/6
4781183/4781183 [==============================] - 48s 10us/sample - loss: 0.6471 - val_loss: 0.7515


In [16]:
# train genre model
genres = model2.fit(
    books.book_id.values, books['adult-fiction'].values,
    batch_size=512, 
    epochs=6,
    validation_split=.2)

Train on 7200 samples, validate on 1800 samples
Epoch 1/6
7200/7200 [==============================] - 1s 99us/sample - loss: 0.6952 - accuracy: 0.4913 - AUC: 0.4953 - val_loss: 0.6892 - val_accuracy: 0.5500 - val_AUC: 0.5699
Epoch 2/6
7200/7200 [==============================] - 0s 7us/sample - loss: 0.6897 - accuracy: 0.5326 - AUC: 0.5474 - val_loss: 0.6840 - val_accuracy: 0.5900 - val_AUC: 0.6140
Epoch 3/6
7200/7200 [==============================] - 0s 7us/sample - loss: 0.6856 - accuracy: 0.5525 - AUC: 0.5756 - val_loss: 0.6805 - val_accuracy: 0.5911 - val_AUC: 0.6248
Epoch 4/6
7200/7200 [==============================] - 0s 7us/sample - loss: 0.6823 - accuracy: 0.5654 - AUC: 0.5914 - val_loss: 0.6775 - val_accuracy: 0.5939 - val_AUC: 0.6266
Epoch 5/6
7200/7200 [==============================] - 0s 7us/sample - loss: 0.6800 - accuracy: 0.5717 - AUC: 0.5992 - val_loss: 0.6753 - val_accuracy: 0.5944 - val_AUC: 0.6297
Epoch 6/6
7200/7200 [==============================] - 0s 8us/samp

In [17]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

In [18]:
y_test = books_holdout['adult-fiction']
y_score = pd.DataFrame(model2.predict(books_holdout.book_id.values))
y_pred = y_score.round().astype('int')

In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.66      0.60       505
           1       0.57      0.47      0.51       495

    accuracy                           0.56      1000
   macro avg       0.56      0.56      0.56      1000
weighted avg       0.56      0.56      0.56      1000



In [20]:
pd.DataFrame(confusion_matrix(y_test, y_pred))

,0,1
0,332,173
1,264,231


## Save predictions

In [21]:
# add test flag
books['test'] = 0
books_holdout['test'] = 1

# combine train and test together
books = pd.concat((books, books_holdout), ignore_index=True).sort_values('book_id')

In [22]:
pred_df = pd.DataFrame(
    data=model2.predict(books.book_id.values).round().astype('int'),
    columns=['prediction']
)

pred_df = pd.concat(
    (books, pred_df),
    axis=1
)

pred_df[['book_id','adult-fiction','prediction','test']].to_csv(path+'predictions.csv', index=False)

pred_df.head()

,book_id,original_title,language_code,adult,adult-fiction,adventure,book-club,contemporary,fantasy,fiction,mystery,romance,young-adult,test,prediction
0,4896,Second Grave on the Left,en-US,1,1,0,0,1,1,1,1,1,0,0,0
1,4782,Метро 2033,ger,1,0,1,0,1,1,1,1,0,0,0,0
2,1496,The Borrowers Afield,0,0,0,1,0,0,1,1,0,0,1,0,1
3,1957,Destined,eng,0,0,0,0,0,1,1,0,1,1,0,0
4,9171,アンダーグラウンド [Andāguraundo],eng,1,0,0,1,1,0,1,0,0,0,0,1


## Save embeddings

In [24]:
emb = model1.get_layer('embedding')

emb_df = pd.concat(
    (books, pd.DataFrame(emb.embeddings.numpy(), columns=['e'+str(i) for i in range(25)])),
    axis=1
)

emb_df.to_csv(path+'embeddings.csv', index=False)

emb_df.head()

,book_id,original_title,language_code,adult,adult-fiction,adventure,book-club,contemporary,fantasy,fiction,...,e15,e16,e17,e18,e19,e20,e21,e22,e23,e24
0,4896,Second Grave on the Left,en-US,1,1,0,0,1,1,1,...,0.096823,-0.262382,-0.128805,0.000981,-0.149066,-0.227919,0.417547,-0.197259,-0.303189,-0.093484
1,4782,Метро 2033,ger,1,0,1,0,1,1,1,...,-0.047106,-0.048310,-0.010779,-0.263847,-0.204119,-0.180064,-0.042321,0.296343,0.025200,-0.177024
2,1496,The Borrowers Afield,0,0,0,1,0,0,1,1,...,-0.120127,0.231151,0.206554,0.227081,-0.174814,0.111580,0.380537,-0.554914,-0.109517,0.199263
3,1957,Destined,eng,0,0,0,0,0,1,1,...,0.107415,-0.132020,0.094204,-0.330955,-0.085299,0.212272,0.336800,-0.001177,-0.100493,0.063971
4,9171,アンダーグラウンド [Andāguraundo],eng,1,0,0,1,1,0,1,...,0.070945,-0.114857,0.107532,-0.056527,0.291845,0.063105,0.053121,-0.056621,-0.172070,0.155401
